<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/jjc/NBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
datas = []
data = ['me free lottery', 'spam']
datas.append(data)
data = ['free get free you', 'spam']
datas.append(data)
data = ['you free scholarship', 'normal']
datas.append(data)
data = ['free to contact me', 'normal']
datas.append(data)
data = ['you won award', 'normal']
datas.append(data)
data = ['you ticket lottery', 'spam']
datas.append(data)

datas

[['me free lottery', 'spam'],
 ['free get free you', 'spam'],
 ['you free scholarship', 'normal'],
 ['free to contact me', 'normal'],
 ['you won award', 'normal'],
 ['you ticket lottery', 'spam']]

In [35]:
datas = []
data = ['I love you', '긍정']
datas.append(data)
data = ['love happy weekend', '긍정']
datas.append(data)
data = ['bore work job', '부정']
datas.append(data)
data = ['I hate you', '중립']
datas.append(data)
data = ['bore weekend', '부정']
datas.append(data)
data = ['happy together', '긍정']
datas.append(data)

datas

[['I love you', '긍정'],
 ['love happy weekend', '긍정'],
 ['bore work job', '부정'],
 ['I hate you', '중립'],
 ['bore weekend', '부정'],
 ['happy together', '긍정']]

In [132]:
import pandas as pd
import numpy as np

# 데이터프레임은 인덱스에 단어, 칼럼에 타겟
class NBC():
    
    df = pd.DataFrame([])
    k = 0.5

    def __init__(self, **kwargs):
        try:
            self.k = kwargs['k']
        except:
            pass
    
    def add_data(self, datas):
        target_ls = list(set(datas['target']))
        words = {}

        for data in datas.iloc:
            for target in target_ls:
                try :
                    if data['target'] == target:
                        words[target].extend(data['words'].split())
                except:
                    if data['target'] == target:
                        words[target] = data['words'].split()

        t_word = []
        for w in list(words.values()):
            t_word.extend(w)
        
        set_word = list(set(t_word))
        counts = {}
        for sw in set_word:
            for target in target_ls:
                try :
                    counts[target].append(words[target].count(sw))
                except :
                    counts[target] = [words[target].count(sw)]
        
        df = pd.DataFrame(set_word, columns=['words'])
        for word in counts:
            df[word] = counts[word]
        df.set_index('words', inplace = True)
        self.df = df
        return self.df

    def pol_score(self, target, s):
        df = self.df
        k = self.k

        totals = {}
        for i in df:
            totals[i] = df[i].sum()
        
        tokens = s.split()
        total = np.asarray(list(totals.values())).sum()

        logs = {}
        for i in df:            
            logs[i] = np.log(float(totals[i]) / float(total))
        
        for token in tokens:
            for t in df:
                logs[t] += np.log( (k + df[t][token]) / (2*k +totals[t]) )

        bunmo = 0
        for log in logs:
            bunmo += np.exp(logs[log])
        bunja = np.exp(logs[target])
        return bunja / bunmo

In [133]:
import pandas as pd
test = pd.DataFrame(datas, columns = ['words', 'target'])

nbc = NBC()
nbc.add_data(test)

,부정,긍정,중립
words,,,
work,1,0,0
you,0,1,1
happy,0,2,0
bore,2,0,0
love,0,2,0
together,0,1,0
weekend,1,1,0
job,1,0,0
hate,0,0,1


In [134]:
nbc.pol_score('긍정', 'happy weekend')

0.7103218645948945